<a href="https://colab.research.google.com/github/yujungha/Data_Analysis/blob/main/Kakao_Log_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [152]:
def dateformat(y=0,m=0,d=0,h=0,mm=0,s=0):
    return "%d-%02d-%02d %02d:%02d:%02d"%(y,m,d,h,mm,s)

In [ ]:
f = open('Talk_2021.4.29 21_54-1.txt', mode='r', encoding='utf-8')

file_name = f.readline()
save_date_re = re.compile('저장한 날짜 : ([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2})')
m = save_date_re.match(f.readline())
save_date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))

f.readline() # 공백 제거
f.readline() # 공백 제거
f.readline() # 공백 제거

history = []
histline = ''
date = None

date_re = re.compile('([0-9]{4})년 ([0-9]{1,2})월 ([0-9]{1,2})일 ((월|화|수|목|금|토|일)요일)')
invite_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}): ((.+)님이 .+ 초대했습니다.)')
exit_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}): ((.+)님이 나갔습니다.)')
chat_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}), (.+) : (.+)')

chat_temp = ''

i=0
while True:
  line = f.readline()
  if not line:
    break
  # print(line, end='')
 
  # chat
  m = chat_re.match(line)
  if m:
    # print(line, end='')
    date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
    user = m[6]
    content = m[7]
    history.append([date, user, content])
    continue

  # date
  m = date_re.match(line)
  if m:
    date = dateformat(int(m[1]), int(m[2]), int(m[3]))
    # print(line, end='')
    continue

  # invite
  m = invite_re.match(line)
  if m:
    date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
    user = m[7]
    content = m[6]
    history.append([date, user, content])
    continue
# history
  # \n chat
  history[-1][2] += line
df = pd.DataFrame(history, columns=['Date', 'User', 'Message'])
df

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4716 entries, 0 to 4715
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     4716 non-null   object
 1   User     4716 non-null   object
 2   Message  4716 non-null   object
dtypes: object(3)
memory usage: 110.7+ KB


In [ ]:
df2 =  df[False == df.duplicated('User')]
df2

활발한 시간대

In [156]:
def func(x):
  date = x.split(' ')
  time = date[1].split(':')[0]
  return int(time)

In [ ]:
time = df['Date'].apply(func).values
num = np.zeros(24)
for i in range(len(time)):
  num[time[i] - 1] += 1
time, num

In [ ]:
plt.plot(range(1, 25), num)
plt.show()

In [182]:
names = df2.iloc[:3, :]['User'].values
names

array(['19 김은하', '유정하', '누리고라스'], dtype=object)

In [195]:
def active_time(name):
  name_df = df[df['User'] == name]
  time = name_df['Date'].apply(func)
  num = np.zeros(24)
  t = time.values

  for i in range(len(t)):
    num[t[i] - 1] += 1

  # plt.title(name)
  plt.plot(range(1, 25), num)
  plt.show()
  return

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic')

In [ ]:
for i in range(len(names)):
  print(active_time(names[i]))

이름 언급 횟수

In [179]:
def mention(name):
  return len(df[df['Message'].str.contains(name)])

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic')

In [ ]:
new_names = ['은하', '정하', '누리']
num = []
for i in range(len(new_names)):
  num.append(mention(new_names[i]))

plt.bar(new_names, num)
# plt.pie(num, labels=new_names, autopct='%.1f%%')
plt.show()

대화해보기

In [ ]:
a = df['Message'].values
b = []
for i in range(len(a) - 1):
  b.append([a[i], a[i + 1]])
b

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

a = np.array(df[['User', 'Message']].values)
# a = [['19 김은하', '19 김은하님이 누리고라스님과 유정하님을 초대했습니다.'],
#     ['19 김은하', '애들아'],
#     ['19 김은하', '조이름생각해바']]
i = 0
while i != len(a)-1: # 같은 사람이 연속으로 카톡을 보냈을 경우 한 문장으로 만들어줌
  print('%d / %d'%(i, len(a)))
  if a[i][0] == a[i+1][0]:
    a[i][1] += ' ' + a[i+1][1]
    a = np.delete(a, i+1, 0)
  else:
    a[i][1] = spell_checker.check(a[i][1]).checked
    i += 1

In [ ]:
from hanspell import spell_checker

a = np.array(df[['User', 'Message']].values)
# a = [['19 김은하', '19 김은하님이 누리고라스님과 유정하님을 초대했습니다.'],
#     ['19 김은하', '애들아'],
#     ['19 김은하', '조이름생각해바']]
i = 0
while i != len(a) - 1: # 같은 사람이 연속으로 카톡을 보냈을 경우 한 문장으로 만들어줌
  print('%d / %d'%(i, len(a)))
  if a[i][0] == a[i + 1][0]:
    a[i][1] += ' ' + a[i + 1][1]
    a = np.delete(a, i + 1, 0)
  else:
    a[i][1] = spell_checker.check(a[i][1]).checked
    i += 1

a = a.T[1]

b = []
for i in range(len(a) - 1):
  b.append([a[i], a[i + 1]])
b

In [ ]:
import random

while True:
  Q = input("질문 : ")
  l1 = []
  l2 = []
  l3 = []

  for i in range(len(b)):
    if Q == b[i][0]: # 질문과 텍스트 완벽 일치
      # print('[완벽일치]', b[i][0])
      l1.append(b[i][1])
      
    elif b[i][0].find(Q) != -1: # 텍스트에 질문이 포함
      # print('[포함]',b[i][0])
      l2.append(b[i][1])
  
    else: # [포함-2] 채팅
      n = 0
      for q in Q.split(' '):
        if b[i][0].find(q) != -1:
          n += 1
      if n > 0:
        # print('[포함-%d]'%n,b[i][0])
        l3.append(b[i][1])

  if len(l1) != 0:
    print('답변 :',random.choice(l1))
  elif len(l2) != 0:
    print('답변 :',random.choice(l2))
  elif len(l3) != 0:
    print('답변 :',random.choice(l3))
  else:
    print('답변 : 맞아맞아')
  print('\n=================================================\n')